In [9]:
# Necessary imports

import numpy as np
import matplotlib.pyplot as plt

from qiskit.circuit.library import *
from qiskit.providers.aer import *

from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS

from qiskit import QuantumCircuit
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_algorithms.utils import algorithm_globals
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

# Set seed for random generators
algorithm_globals.random_seed = 42

In [1]:
import torch
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from torch.nn import MSELoss

from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN

fm = QuantumCircuit(2)
input_parameters = ParameterVector("x", 2)
fm.ry(input_parameters[0], 0)
fm.ry(input_parameters[1], 1)
fm.cx(0, 1)

ansatz = QuantumCircuit(2)
weights = ParameterVector("w", 4)

ansatz.ry(weights[0], 0)
ansatz.rz(weights[1], 0)
ansatz.ry(weights[2], 1)
ansatz.rz(weights[3], 1)
ansatz.cx(0, 1)

qnn_qc = QuantumCircuit(2)
qnn_qc.compose(fm, inplace=True)
qnn_qc.compose(ansatz, inplace=True)

qnn = EstimatorQNN(circuit=qnn_qc, input_params=input_parameters, weight_params=weights)
model = TorchConnector(qnn)

features = torch.rand((20, 2))
labels = torch.randint(0, 2, (20, 1)) * 2. - 1

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = MSELoss()

for i in range(10):
    optimizer.zero_grad()
    loss = criterion(model(features), labels)
    print(f"Iteration: {i}, loss: {loss.item()}")
    loss.backward()
    optimizer.step()

Iteration: 0, loss: 2.343188762664795
Iteration: 1, loss: 2.2014060020446777
Iteration: 2, loss: 1.9895824193954468
Iteration: 3, loss: 1.7329108715057373
Iteration: 4, loss: 1.467564344406128
Iteration: 5, loss: 1.2330716848373413
Iteration: 6, loss: 1.0632938146591187
Iteration: 7, loss: 0.9754740595817566
Iteration: 8, loss: 0.9627001881599426
Iteration: 9, loss: 0.9981807470321655


In [10]:
simulator_gpu = AerSimulator()
simulator_gpu.set_options(device='GPU')

In [11]:
# Define and create QNN
def create_qnn():
    feature_map = ZZFeatureMap(2)
    ansatz = RealAmplitudes(2, reps=1)
    qc = QuantumCircuit(2)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # REMEMBER TO SET input_gradients=True FOR ENABLING HYBRID GRADIENT BACKPROP
    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn


qnn4 = create_qnn()

